In [7]:
import numpy as np
import pandas as pd

#import data
df = pd.read_csv('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/data_npj/Graph_data.csv')

In [2]:
df

,Type,Model,Error_E,Error_F,Cost
0,GM-NN (CPU),593-512-512-1,2.32,43,43.8100
1,GM-NN (CPU),593-128-128-1,2.42,46,40.4200
2,GM-NN (CPU),593-32-32-1,2.56,48,37.5400
3,GM-NN (CPU),360-512-512-1,2.79,51,44.2500
4,GM-NN (CPU),910-512-512-1,2.14,42,50.0400
5,GM-NN (GPU),593-512-512-1,2.32,43,7.2500
6,GM-NN (GPU),593-128-128-1,2.42,46,7.4700
7,GM-NN (GPU),593-32-32-1,2.56,48,7.2800
8,GM-NN (GPU),360-512-512-1,2.79,51,7.3000
9,GM-NN (GPU),910-512-512-1,2.14,42,7.2300


In [8]:
def calculate_rel_e(df:pd.DataFrame):
    '''Calculate normalized error. Returns dataframe with new column ["Normalized error"].'''
    df['Normalized error']=0
    for (kind, model, error_e, error_f) in zip(df['Type'],df['Model'],df['Error_E'],df['Error_F']):
        df.loc[(df['Type']==kind)&(df['Model']==model),'Normalized error']=(1/2)*( error_f/max(df['Error_F']) + error_e/max(df['Error_E']) )
    return df

In [9]:
df=calculate_rel_e(df)

In [10]:
import plotly.express as px

fig = px.scatter(df, x='Cost', y='Normalized error', color='Type', text='Model', log_x=True)
fig.update_layout(
    xaxis_title=dict(text='Computational cost of inference [<i>μs/atom/step</i>]')
) 
#fig.update_layout(legend=dict(
#    yanchor
#))

In [11]:
import plotly.graph_objects as go

layout = go.Layout(
    yaxis1=dict(domain=[0.,0.7]),
    yaxis2=dict(domain=[0.75,1])
)

# add assignment to y subplot
df['y axes']='y1'
df.loc[(df['Normalized error']>0.2),'y axes']='y2'

fig = go.Figure(data=[], layout=layout)

for kind in df['Type'].unique():
    subset=df.query('Type==@kind')
    fig.add_trace(go.Scatter(
        x=subset['Cost'],
        y=subset['Normalized error'],
        mode='markers',
        name=kind,
        hovertemplate="%{text}"+"<br>Computational cost: %{x} μs"+"<br>Normalized error: %{y}",
        text=subset['Model'],
        legendgroup=kind,
        yaxis=subset['y axes'].unique()[0],
    ),
    )
fig.update_xaxes(title_text='Computational cost of inference [<i>μs/atom/step</i>]',type="log",minor_ticks='inside')
fig.update_layout(yaxis1={'title':'Normalized error','minor_ticks':'inside'},yaxis2={'minor_ticks':'inside'},grid_yaxes=['y1']) #,'position':0